# BB84 QKD Parameters Optimization

In [1]:
import torch

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use MPS as the device
    print("Using MPS for GPU acceleration.")
else:
    device = torch.device("cpu")  # Fallback to CPU
    print("MPS is not available. Using CPU.")

Using MPS for GPU acceleration.


## Imports

In [2]:
from math import exp, factorial  # For basic math operations
from scipy.optimize import minimize, dual_annealing, differential_evolution, Bounds  # For optional optimization methods
import matplotlib.pyplot as plt  # For plotting
import numpy as np  # For tensor operations if needed alongside PyTorch
from tqdm import tqdm  # For progress bars
from joblib import Parallel, delayed  # For parallel processing
import os  # For file system operations
import json  # For saving and loading datasets
import time  # For timing operations
import pandas as pd  # For data manipulation
from tabulate import tabulate  # For pretty-printing tables

import os
import sys

# Get the notebook's directory
notebook_dir = os.getcwd()
# Add parent directory to path
project_root = os.path.dirname(notebook_dir)
sys.path.append(project_root)


# PyTorch imports
import torch  # Core PyTorch library

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Experimental Parameters

In [4]:
# e_1
# Fiber lengths
Ls = torch.linspace(5, 200, 1000)  # Fiber lengths in km
L_BC = Ls
e_1 = L_BC / 100

#e_2
P_dc_value = torch.tensor(6e-7, device=device)   # Dark count probability
Y_0 = P_dc_value
# 2.7*10** -7
# P_dc = 6 * 10 ** (-7)   # given in the paper, discussed with range from 10^-8 to 10^-5
e_2 = -torch.log(Y_0)

# e_3
# Misalignment error probability
# 4*1e-2          # given in the paper, discussed with range from 0 to 0.1
e_mis = torch.tensor(5e-3, device=device)  # Misalignment error probability # given in the paper, discussed with range from 0 to 0.1 
e_d = e_mis
# 0.026 
e_3 = e_d * 100

# e_4
# Detected events
n_X_values = torch.tensor([10 ** s for s in range(4, 15)], dtype=torch.float64, device=device)  # Detected events # Detected events
# n_X_values = torch.tensor([10**s for s in range(6, 11)], dtype=torch.int64)
N = n_X_values
e_4 = torch.log10(N)

# Prepare input combinations
# inputs = [(L, n_X) for L in torch.linspace(0.1, 200, 100) for n_X in n_X_values]

## Other Parameters

In [5]:
alpha = 0.2  # Attenuation coefficient (dB/km), given in the paper
eta_Bob = 0.1  # Detector efficiency, given in the paper
P_ap = 0  # After-pulse probability
f_EC = 1.16  # Error correction efficiency
# secutity error 
epsilon_sec = 1e-10 # is equal to kappa * secrecy length Kl, range around 1e-10 Scalar, as it is a single value throughout the calculations.
# correlation error
epsilon_cor = 1e-15 # given in the paper, discussed with range from 0 to 10e-10
# Dark count probability
n_event = 1  # for single photon event
# Misalignment error probability
# 4*1e-2          # given in the paper, discussed with range from 0 to 0.1
kappa = 1e-15           # given in the paper
f_EC = 1.16             # given in the paper, range around 1.1


## Optimal Paramters

In [6]:
def optimal_parameters(params):
    mu_1, mu_2, P_mu_1, P_mu_2, P_X_value = params
    mu_3 = 2e-4
    P_mu_3 = 1 - P_mu_1 - P_mu_2
    P_Z_value = 1 - P_X_value
    mu_k_values = torch.tensor([mu_1, mu_2, mu_3])
    return params, mu_3, P_mu_3, P_Z_value, mu_k_values

In [9]:
from QKD_Functions.QKD_Functions_Torch import (
    calculate_factorial,
    calculate_tau_n,
    calculate_eta_ch,
    calculate_eta_sys,
    calculate_D_mu_k,
    calculate_n_X_total,
    calculate_N,
    calculate_n_Z_total,
    calculate_e_mu_k,
    calculate_e_obs,
    calculate_h,
    calculate_lambda_EC,
    calculate_sqrt_term,
    calculate_tau_n,
    calculate_n_pm, 
    calculate_S_0,
    calculate_S_1,
    calculate_m_mu_k,
    calculate_m_pm,
    calculate_v_1,
    calculate_gamma,
    calculate_Phi,
    calculate_LastTwoTerm,
    calculate_l,
    calculate_R,
    experimental_parameters,
    other_parameters,
    calculate_key_rates_and_metrics,
    penalty, 
    objective,
# objective_with_logging
)

In [8]:
# Bounds for optimization parameters
bounds = [
    (1*1e-6, 1),  # mu_1, if it represents a mean photon number, adjust max as needed
    (1*1e-6, 1),  # mu_2
    (1*1e-6, 1),   # P_mu_1, probability
    (1*1e-6, 1),   # P_mu_2, probability
    (1*1e-6, 1),   # P_X_value, probability
]

In [9]:
def optimize_single_instance(input_params, bounds, alpha, eta_Bob, P_dc_value, epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event, max_retries=20):
    """
    Optimize key rates using PyTorch optimization
    """
    # Move inputs to device
    L, n_X = input_params
    L = torch.tensor(L, dtype=torch.float64, device=device)
    n_X = torch.tensor(n_X, dtype=torch.float64, device=device)
    
    best_key_rate = float('-inf')
    best_params = None

    for attempt in range(max_retries):
        try:
            # Initialize parameters with random values within bounds
            params = []
            for (lower, upper) in bounds:
                param = torch.rand(1, dtype=torch.float64, device=device) * (upper - lower) + lower
                param.requires_grad_(True)
                params.append(param)

            # Use Adam optimizer
            optimizer = torch.optim.Adam(params, lr=0.01)
            
            # Optimization loop
            for i in range(500):  # max iterations
                optimizer.zero_grad()
                
                # Compute key rate
                current_params = [p.item() for p in params]
                key_rate = objective(current_params, L, n_X, alpha, eta_Bob, P_dc_value, 
                                  epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event)[0]
                
                # Convert to tensor if not already
                if not isinstance(key_rate, torch.Tensor):
                    key_rate = torch.tensor(key_rate, dtype=torch.float64, device=device)
                
                # Negative for minimization
                loss = -key_rate
                
                # Backward pass
                loss.backward()
                
                # Update parameters
                optimizer.step()
                
                # Project parameters back to bounds
                with torch.no_grad():
                    for param, (lower, upper) in zip(params, bounds):
                        param.clamp_(lower, upper)
                
                # Check convergence
                if i > 0 and abs(prev_loss - loss.item()) < 1e-6:
                    break
                prev_loss = loss.item()

            # Get optimized parameters and key rate
            optimized_params = [p.item() for p in params]
            optimized_key_rate = -loss.item()

            # Check if key rate is valid
            if torch.log10(torch.tensor(max(optimized_key_rate, 1e-10))) > 0:
                print(f"Attempt {attempt + 1}: Abnormal key rate detected (log10 > 0). Retrying...")
                continue

            # Update best results if improved
            if optimized_key_rate > best_key_rate:
                best_key_rate = optimized_key_rate
                best_params = optimized_params

            # Check if parameters have stabilized
            if best_params is not None and torch.allclose(
                torch.tensor(best_params, device=device),
                torch.tensor(optimized_params, device=device),
                atol=1e-4
            ):
                break

        except Exception as e:
            print(f"Attempt {attempt + 1}: Optimization error: {e}. Retrying...")

    if best_params is not None:
        return L.item(), n_X.item(), best_key_rate, best_params

    print(f"Optimization failed after {max_retries} retries.")
    return L.item(), n_X.item(), float('nan'), [float('nan')] * len(bounds)

In [10]:
# Define input parameters for a single instance
single_input = (Ls[0].item(), n_X_values[0])  # Example: first fiber length and first n_X value

# Measure start time
start_time = time.time()

# Run the optimization for the single instance
L, n_X, optimized_key_rate, optimized_params = optimize_single_instance(  # Note the _torch suffix
    single_input, bounds, alpha, eta_Bob, P_dc_value, epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event
)

# Measure end time
end_time = time.time()

# Output the results with parameter names
parameter_names = ["mu_1", "mu_2", "P_mu_1", "P_mu_2", "P_X_value"]
optimized_parameters = {name: value for name, value in zip(parameter_names, optimized_params)}

print(f"Optimization for a single instance took {end_time - start_time:.2f} seconds.")
print(f"Fiber Length: {L} km, Detected Events (n_X): {n_X}")
print(f"Optimized Key Rate: {optimized_key_rate:.3e}")
print("Optimized Parameters:")
for name, value in optimized_parameters.items():
    print(f"  {name}: {value:.6f}")

/var/folders/8m/0wg1hssn6n79tjc8mh6p_spr0000gn/T/ipykernel_3901/3589318213.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  n_X = torch.tensor(n_X, dtype=torch.float64, device=device)


Attempt 1: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 2: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 3: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 4: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 5: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 6: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 7: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 8: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 9: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 10: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 11: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 12: Optimization error: 'float' object has no attribute 'to'. Retrying...
Attempt 13: Optimization 

In [11]:
def generate_comprehensive_dataset(Ls, n_X_values, bounds, alpha, eta_Bob, P_dc_value, epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event, n_jobs=12):
    """
    Generate a comprehensive dataset using parallel processing with PyTorch
    """
    # Create all combinations of L and n_X
    combinations = [(L.item(), float(n_X)) for L in Ls for n_X in n_X_values]
    
    # Initialize dictionary to store results
    categorized_dataset = {float(n_X): [] for n_X in n_X_values}
    
    def process_single_combination(combo):
        """Process a single (L, n_X) combination"""
        L, n_X_float = combo
        
        # Optimize parameters for this combination
        result = optimize_single_instance(
            (L, n_X_float), bounds, alpha, eta_Bob, P_dc_value, 
            epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event
        )
        
        if result is None or result[2] <= 0:  # Skip if optimization failed or key rate is invalid
            return None
            
        L_val, _, penalized_key_rate, optimized_params = result
        mu_1, mu_2, P_mu_1, P_mu_2, P_X_value = optimized_params
        
        # Compute normalized parameters using PyTorch
        return {
            "n_X": n_X_float,
            "fiber_length": float(L_val),
            "e_1": float(L_val / 100),
            "e_2": float(-torch.log10(torch.tensor(P_dc_value, device=device))),
            "e_3": float(e_mis * 100),
            "e_4": float(torch.log10(torch.tensor(n_X_float, device=device))),
            "key_rate": float(max(penalized_key_rate, 1e-10)),
            "optimized_params": {
                "mu_1": float(mu_1),
                "mu_2": float(mu_2),
                "P_mu_1": float(P_mu_1),
                "P_mu_2": float(P_mu_2),
                "P_X_value": float(P_X_value)
            }
        }
    
    # ... rest of the function remains the same ...

# Usage with PyTorch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Define parameter space with PyTorch
Ls = torch.linspace(5, 200, 1000, device=device)  # 1000 points from 5 to 200 km
n_X_values = [10**s for s in range(4, 15)]  # n_X from 10^4 to 10^14

# Generate the comprehensive dataset with parallel processing
dataset = generate_comprehensive_dataset(
    Ls, n_X_values, bounds, alpha, eta_Bob, P_dc_value, 
    epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event,
    n_jobs=12
)

Using device: mps


Reasonableness Check for Execution Time:

A single optimization took 1.81 seconds. For 10,000 instances, the time would scale proportionally if no optimizations (e.g., batch parallelism) are applied:

$ \text{Total Time} = 1.81 \times 10,000 \approx 5.03 \, \text{hours}$ 

With 12 CPU cores using joblib, the time should reduce by approximately  1 / 12 :

$ \text{Total Time with 12 CPUs} \approx 5.03 / 12 \approx 25.13 \, \text{minutes}$


## Parallel Dataset Generation Using joblib

In [12]:
def generate_dataset(Ls, n_X_values, bounds, alpha, eta_Bob, P_dc_value, epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event, device=None):
    """
    Generate a dataset by optimizing key rates for various fiber lengths and n_X values.
    """
    # Create input combinations
    inputs = [(L.item(), n_X.item()) for L in Ls for n_X in n_X_values]

    print("Generating dataset...")
    results = []

    def process_batch(batch):
        batch_results = []
        for input_params in batch:
            L, n_X, penalized_key_rate, optimized_params = optimize_single_instance_on_gpu(
                input_params, bounds, alpha, eta_Bob, P_dc_value, epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event
            )
            batch_results.append((L, n_X, penalized_key_rate, optimized_params))
        return batch_results

    # Split inputs into batches
    batch_size = 100
    batches = [inputs[i:i+batch_size] for i in range(0, len(inputs), batch_size)]


    # Progress bar for monitoring
    with tqdm(total=len(batches), desc="Generating Dataset") as progress_bar:
        results = Parallel(n_jobs=12)(
            delayed(process_batch)(batch) for batch in batches
        )
        progress_bar.update(len(batches))

    # Flatten results
    results = [item for sublist in results for item in sublist]

    # Process results into a dataset
    dataset = []
    for L, n_X, penalized_key_rate, optimized_params in results:
        # Extract optimized parameters (ensure the order matches the bounds setup)
        mu_1, mu_2, P_mu_1, P_mu_2, P_X_value = optimized_params

        # Compute normalized parameters
        e_1 = L / 100  # Normalize fiber length
        e_2 = -torch.log10(P_dc_value).item()  # Normalize dark count probability
        e_3 = (e_mis * 100).item()  # Normalize misalignment error probability
        e_4 = torch.log10(torch.tensor(n_X, dtype=torch.float64, device=device)).item()  # Normalize number of pulses

        # Append processed data
        dataset.append({
            "e_1": e_1,
            "e_2": e_2,
            "e_3": e_3,
            "e_4": e_4,
            "key_rate": penalized_key_rate,
            "optimized_params": {
                "mu_1": mu_1,
                "mu_2": mu_2,
                "P_mu_1": P_mu_1,
                "P_mu_2": P_mu_2,
                "P_X_value": P_X_value,
            }
        })

    return dataset

In [13]:
# Experiment-specific parameters

# Define parameters as PyTorch tensors
P_dc_value = torch.tensor(6e-7, device=device, dtype=torch.float64)  # Dark count probability
e_mis = torch.tensor(5e-3, device=device, dtype=torch.float64)       # Misalignment error probability
alpha = torch.tensor(0.2, device=device, dtype=torch.float64)        # Attenuation coefficient (dB/km), given in the paper
eta_Bob = torch.tensor(0.1, device=device, dtype=torch.float64)      # Detector efficiency, given in the paper
P_ap = torch.tensor(1e-6, device=device, dtype=torch.float64)        # After-pulse probability # 4*1e-2          # given in the paper, discussed with range from 0 to 0.1
f_EC = torch.tensor(1.16, device=device, dtype=torch.float64)        # Error correction efficiency # given in the paper, range around 1.1
epsilon_sec = torch.tensor(1e-10, device=device, dtype=torch.float64) # Security error # is equal to kappa * secrecy length Kl, range around 1e-10 Scalar, as it is a single value throughout the calculations.
epsilon_cor = torch.tensor(1e-15, device=device, dtype=torch.float64) # Correlation error
n_event = torch.tensor(1, device=device, dtype=torch.float64)         # For single photon event
kappa = torch.tensor(1e-15, device=device, dtype=torch.float64)       # given in the papere_1 # given in the paper, discussed with range from 0 to 10e-10

# Define parameter space
Ls = torch.linspace(0.1, 220, 100, device=device, dtype=torch.float64)  # Fiber lengths
n_X_values = torch.logspace(6, 10, 8, device=device, dtype=torch.float64)  # Detected events


# 
# torch.linspace(0.1, 220, 10):
# 	•	Generates 10 equally spaced values between 0.1 and 220. For example: [0.1, 27.88, 55.66, ..., 220].
# 	•	Represents 10 fiber lengths to evaluate, rather than 1000.
# np.logspace(6, 8, 5):
# 	•	Generates 5 logarithmically spaced values between 10^6 and 10^8. For example: [1e6, 3.16e6, 1e7, ..., 1e8].
# 	•	Represents the number of detected events (n_X) in a smaller range with fewer steps.

import cProfile
cProfile.run("""
optimize_single_instance(
    (Ls[0].item(), n_X_values[0].item()), bounds, alpha, eta_Bob, P_dc_value, epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event, device=device
)
""")


# Measure total dataset generation time
start_time = time.time()

dataset = generate_dataset(
    Ls, n_X_values, bounds, alpha, eta_Bob, P_dc_value, epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event, device=device
)

end_time = time.time()
print(f"Dataset generation completed in {(end_time - start_time) / 60:.2f} minutes.")

# Save to JSON
output_filename = "training_dataset.json"
with open(output_filename, "w") as f:
    json.dump(dataset, f)
print(f"Dataset saved as '{output_filename}'.")

TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.

In [ ]:
import torch
import matplotlib.pyplot as plt
import json

# Load the dataset
with open("training_dataset.json", "r") as f:
    data = json.load(f)

# Extract fiber lengths and key rates
e_1 = torch.tensor([item["e_1"] * 100 for item in data], dtype=torch.float64)  # Denormalize fiber lengths (convert to km)
key_rate = torch.tensor([item["key_rate"] for item in data], dtype=torch.float64)  # Correct key name

# Extract optimized parameters
mu_1 = torch.tensor([item["optimized_params"]["mu_1"] for item in data], dtype=torch.float64)  # Access nested keys
mu_2 = torch.tensor([item["optimized_params"]["mu_2"] for item in data], dtype=torch.float64)
P_mu_1 = torch.tensor([item["optimized_params"]["P_mu_1"] for item in data], dtype=torch.float64)
P_mu_2 = torch.tensor([item["optimized_params"]["P_mu_2"] for item in data], dtype=torch.float64)
P_X_value = torch.tensor([item["optimized_params"]["P_X_value"] for item in data], dtype=torch.float64)

# Sort by fiber length for smooth plotting
sorted_indices = torch.argsort(e_1)
e_1_sorted = e_1[sorted_indices]
key_rate_sorted = key_rate[sorted_indices]
mu_1_sorted = mu_1[sorted_indices]
mu_2_sorted = mu_2[sorted_indices]
P_mu_1_sorted = P_mu_1[sorted_indices]
P_mu_2_sorted = P_mu_2[sorted_indices]
P_X_value_sorted = P_X_value[sorted_indices]

# Plot the data
plt.figure(figsize=(15, 6))

# Left plot: Penalized Key Rate
plt.subplot(1, 2, 1)
plt.plot(e_1_sorted.cpu(), torch.log10(torch.clamp(key_rate_sorted, min=1e-10)).cpu(), label="Penalized Key Rate (log10)")
plt.xlabel("Fiber Length (km)")
plt.ylabel("log10(Penalized Key Rate)")
plt.title("Penalized Key Rate vs Fiber Length")
plt.grid(True, linestyle='--', linewidth=0.5)
plt.legend()

# Right plot: Optimized Parameters
plt.subplot(1, 2, 2)
plt.plot(e_1_sorted.cpu(), mu_1_sorted.cpu(), label="mu_1")
plt.plot(e_1_sorted.cpu(), mu_2_sorted.cpu(), label="mu_2")
plt.plot(e_1_sorted.cpu(), P_mu_1_sorted.cpu(), label="P_mu_1")
plt.plot(e_1_sorted.cpu(), P_mu_2_sorted.cpu(), label="P_mu_2")
plt.plot(e_1_sorted.cpu(), P_X_value_sorted.cpu(), label="P_X_value")
plt.xlabel("Fiber Length (km)")
plt.ylabel("Optimized Parameters")
plt.title("Optimized Parameters vs Fiber Length")
plt.grid(True, linestyle='--', linewidth=0.5)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
with open("training_dataset.json", "r") as f:
    data = json.load(f)

print(data[0])  # Print the first item to inspect its structure

In [ ]:
import json

# Load the dataset
with open("training_dataset.json", "r") as f:
    data = json.load(f)

# Extract the top 100 entries
top_100_entries = data[:100]

# Display the top 100 entries
print("\nTop 100 entries:")
for idx, entry in enumerate(top_100_entries, 1):
    print(f"Entry {idx}: {entry}")

# Optional: Flatten the JSON structure and save as a CSV file
df = pd.json_normalize(data, sep='_')  # Flatten the JSON structure

# Convert numeric data to PyTorch tensors where applicable
for col in df.select_dtypes(include=['float', 'int']).columns:
    df[col] = df[col].apply(lambda x: torch.tensor(x, dtype=torch.float64))

# Save the DataFrame as a CSV file
output_csv_file = "training_dataset.csv"
df.to_csv(output_csv_file, index=False)

print(f"\nCSV file saved as '{output_csv_file}'.")

# import json
# import pandas as pd

# # Load the JSON data
# with open("training_dataset.json", "r") as f:
#     data = json.load(f)

# # Flatten the JSON structure
# df = pd.json_normalize(data, sep='_')

# # Save the DataFrame as a CSV file
# output_csv_file = "training_dataset.csv"
# df.to_csv(output_csv_file, index=False)

# print(f"CSV file saved as {output_csv_file}.")

In [19]:
# bounds = Bounds([1e-6] * 5, [1.0] * 5)  # Example bounds
# input_params = (100, 1e8)  # Example fiber length and n_X
# result = optimize_single_instance(
#     input_params, bounds, alpha, eta_Bob, P_dc_value, epsilon_sec, epsilon_cor, f_EC, e_mis, P_ap, n_event
# )
# print(result)

## Current Setup

49% of 10,000 iterations completed in 15 minutes. 

Processing speed: 5.16 iterations per second (it/s). 

Total Time Estimation (Current Setup): \
Total iterations: 10,000. \
Completed iterations:  10,000 \times 0.49 = 4,900 . \
Time to complete 4,900 iterations: 15 minutes (900 seconds). \
Estimated total time for 10,000 iterations: 

$\text{Total time} = \frac{\text{Total iterations}}{\text{Processing speed}} = \frac{10,000}{5.16} \approx 1,937 \text{ seconds (32 minutes)}$

So, approximately 32 minutes total is needed for the dataset generation with your current setup.

## Multiprocessing

Assumption:
12 CPU cores available (based on earlier discussions). \
Multiprocessing scales linearly with cores (ideal case, no overhead). 

Parallel Speed Calculation:

If multiprocessing scales ideally: 

$\text{Parallel speed} = \text{Single-threaded speed} \times \text{Number of cores}$


$\text{Parallel speed} = 5.16 \, \text{it/s} \times 12 \approx 61.92 \, \text{it/s}$


Parallel Time Calculation:


$\text{Total time (parallel)} = \frac{\text{Total iterations}}{\text{Parallel speed}} = \frac{10,000}{61.92} \approx 161.5 \, \text{seconds (2.7 minutes)}.$



# Reference
1. https://machinelearningmastery.com/dual-annealing-optimization-with-python/
2. https://en.wikipedia.org/wiki/Global_optimization
3. https://docs.scipy.org/doc/scipy/tutorial/optimize.html